In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
=================================================================
Active Learning Visualisierungen - Deutsche Version mit angepasster Skalierung
=================================================================
Notebook zur Neuerstellung der Plots mit:
- Deutschen Beschriftungen
- Automatisch angepasster Y-Achsen-Skalierung
- Verbesserter Sichtbarkeit kleiner Unterschiede
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Matplotlib konfigurieren für deutsche Umlaute
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# Style setzen
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    try:
        plt.style.use('seaborn-whitegrid')
    except:
        plt.style.use('ggplot')

print("Active Learning Visualisierungen - Deutsche Version")
print("="*60)

# -------------------------------------------------------------------------------
# 1. Daten laden
# -------------------------------------------------------------------------------
print("Lade Ergebnisse...")

try:
    # Hauptergebnisse
    results_df = pd.read_csv('results/mnist_active_learning_results.csv')
    print(f"✓ Hauptergebnisse geladen: {len(results_df)} Einträge")
    
    # Statistische Analyse
    stat_results = pd.read_csv('results/mnist_statistical_analysis.csv')
    print(f"✓ Statistische Analyse geladen: {len(stat_results)} Vergleiche")
    
    # Label-Einsparungen
    savings_df = pd.read_csv('results/mnist_label_savings.csv')
    print(f"✓ Label-Einsparungen geladen: {len(savings_df)} Einträge")
    
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")
    print("Stelle sicher, dass die CSV-Dateien im 'results' Ordner vorhanden sind!")

# -------------------------------------------------------------------------------
# 2. Hilfsfunktionen für deutsche Übersetzungen
# -------------------------------------------------------------------------------

def translate_classifier(name):
    """Übersetzt Klassifikator-Namen ins Deutsche"""
    translations = {
        'CNN': 'CNN',
        'Naive Bayes': 'Naive Bayes',
        'Random Forest': 'Random Forest',
        'Logistic Regression': 'Logistische Regression',
        'SVM': 'SVM'
    }
    return translations.get(name, name)

def translate_strategy(name):
    """Übersetzt Strategie-Namen ins Deutsche"""
    translations = {
        'Random Sampling': 'Zufällige Auswahl',
        'Entropy Sampling': 'Entropie-Auswahl',
        'Margin Sampling': 'Margin-Auswahl',
        'Least Confidence': 'Geringste Konfidenz'
    }
    return translations.get(name, name)

def translate_effect_size(effect):
    """Übersetzt Effektstärke ins Deutsche"""
    translations = {
        'negligible': 'vernachlässigbar',
        'small': 'klein',
        'medium': 'mittel',
        'large': 'groß'
    }
    return translations.get(effect, effect)

# -------------------------------------------------------------------------------
# 3. Funktion zur automatischen Y-Achsen-Anpassung
# -------------------------------------------------------------------------------

def calculate_y_limits(data, margin=0.1):
    """
    Berechnet optimale Y-Achsen-Limits basierend auf den Daten
    
    Args:
        data: Array oder Liste mit Y-Werten
        margin: Relativer Abstand (Standard: 10%)
    
    Returns:
        (y_min, y_max): Optimierte Achsengrenzen
    """
    if len(data) == 0:
        return 0, 1
    
    data_min = np.min(data)
    data_max = np.max(data)
    data_range = data_max - data_min
    
    # Wenn der Bereich sehr klein ist, erweitern
    if data_range < 0.01:
        center = (data_min + data_max) / 2
        y_min = center - 0.01
        y_max = center + 0.01
    else:
        # Normaler Fall: Margin hinzufügen
        y_min = data_min - margin * data_range
        y_max = data_max + margin * data_range
    
    # Sicherstellen, dass wir nicht unter 0 gehen bei Accuracy
    if data_min > 0.5:  # Wahrscheinlich Accuracy-Werte
        y_min = max(0, y_min)
        y_max = min(1, y_max)
    
    return y_min, y_max

# -------------------------------------------------------------------------------
# 4. Lernkurven pro Klassifikator mit angepasster Skalierung
# -------------------------------------------------------------------------------

def plot_learning_curves_german(results_df, stat_results, output_dir='plots'):
    """Erstellt Lernkurven mit deutschen Beschriftungen und optimaler Skalierung"""
    
    # Farben für Strategien
    strategy_colors = {
        'Random Sampling': '#808080',
        'Entropy Sampling': '#1f77b4',
        'Margin Sampling': '#ff7f0e',
        'Least Confidence': '#2ca02c'
    }
    
    classifiers = sorted(results_df['classifier'].unique())
    budget_percentages = sorted(results_df['budget_pct'].unique())
    
    for classifier in classifiers:
        fig, axes = plt.subplots(1, len(budget_percentages), figsize=(20, 5))
        
        if len(budget_percentages) == 1:
            axes = [axes]
        
        # Deutscher Titel
        fig.suptitle(f'{translate_classifier(classifier)} - Active Learning Leistung mit statistischer Signifikanz', 
                     fontsize=16, y=1.02)
        
        # Sammle alle Y-Werte für globale Skalierung
        all_accuracies = []
        
        for budget_idx, budget_pct in enumerate(budget_percentages):
            ax = axes[budget_idx]
            budget_accuracies = []
            
            for strategy in strategy_colors.keys():
                strategy_data = results_df[
                    (results_df['classifier'] == classifier) & 
                    (results_df['strategy'] == strategy) & 
                    (results_df['budget_pct'] == budget_pct)
                ]
                
                if not strategy_data.empty:
                    accuracies = strategy_data['accuracy'].values
                    budget_accuracies.extend(accuracies)
                    all_accuracies.extend(accuracies)
                    
                    mean_acc = np.mean(accuracies)
                    std_acc = np.std(accuracies)
                    
                    # Prüfe Signifikanz
                    is_significant = False
                    effect_size = ""
                    if strategy != 'Random Sampling' and not stat_results.empty:
                        sig_data = stat_results[
                            (stat_results['classifier'] == classifier) & 
                            (stat_results['strategy'] == strategy) & 
                            (stat_results['budget_pct'] == budget_pct)
                        ]
                        if not sig_data.empty:
                            is_significant = sig_data.iloc[0]['significant']
                            effect_size = translate_effect_size(sig_data.iloc[0]['effect_size'])
                    
                    # Label mit Signifikanz
                    label = translate_strategy(strategy)
                    if is_significant:
                        label += f" *({effect_size})"
                    
                    # Balkendiagramm statt Linie für bessere Sichtbarkeit
                    x_pos = list(strategy_colors.keys()).index(strategy)
                    ax.bar(x_pos, mean_acc, yerr=std_acc, 
                          label=label, 
                          color=strategy_colors[strategy],
                          alpha=0.8,
                          capsize=5,
                          width=0.8)
            
            # Optimale Y-Achsen-Skalierung für dieses Budget
            if budget_accuracies:
                y_min, y_max = calculate_y_limits(budget_accuracies, margin=0.05)
                ax.set_ylim(y_min, y_max)
            
            # Deutsche Achsenbeschriftung
            ax.set_xlabel('Strategie', fontsize=11)
            ax.set_ylabel('Test-Genauigkeit', fontsize=11)
            ax.set_title(f'Budget: {int(budget_pct*100)}%', fontsize=12)
            ax.set_xticks(range(len(strategy_colors)))
            ax.set_xticklabels([translate_strategy(s) for s in strategy_colors.keys()], 
                              rotation=45, ha='right')
            
            # Grid
            ax.grid(True, alpha=0.3, axis='y')
            
            # Legende nur beim ersten Plot
            if budget_idx == 0:
                ax.legend(loc='upper right', fontsize=9, framealpha=0.9)
        
        # Signifikanz-Erklärung auf Deutsch
        fig.text(0.5, -0.08, 
                '* = statistisch signifikant (p < 0.05 mit Bonferroni-Korrektur); ' +
                'Effektstärke in Klammern (vernachlässigbar/klein/mittel/groß)',
                ha='center', fontsize=10, style='italic')
        
        plt.tight_layout()
        
        # Speichern
        filename = f'{output_dir}/{classifier.lower().replace(" ", "_")}_lernkurven_deutsch.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"✓ Lernkurven für {classifier} erstellt: {filename}")
        plt.close()

# -------------------------------------------------------------------------------
# 5. Verbesserung gegenüber Zufallsauswahl
# -------------------------------------------------------------------------------

def plot_improvement_german(results_df, output_dir='plots'):
    """Zeigt die prozentuale Verbesserung gegenüber der Zufallsauswahl"""
    
    improvements = []
    classifiers = sorted(results_df['classifier'].unique())
    strategies = ['Entropy Sampling', 'Margin Sampling', 'Least Confidence']
    budget_percentages = sorted(results_df['budget_pct'].unique())
    
    for classifier in classifiers:
        for strategy in strategies:
            for budget_pct in budget_percentages:
                # Baseline: Random Sampling
                random_data = results_df[
                    (results_df['classifier'] == classifier) & 
                    (results_df['strategy'] == 'Random Sampling') & 
                    (results_df['budget_pct'] == budget_pct)
                ]
                
                # Active Learning Strategie
                strategy_data = results_df[
                    (results_df['classifier'] == classifier) & 
                    (results_df['strategy'] == strategy) & 
                    (results_df['budget_pct'] == budget_pct)
                ]
                
                if not random_data.empty and not strategy_data.empty:
                    random_acc = random_data['accuracy'].mean()
                    strategy_acc = strategy_data['accuracy'].mean()
                    
                    # Prozentuale Verbesserung
                    improvement = ((strategy_acc - random_acc) / random_acc) * 100 if random_acc > 0 else 0
                    
                    improvements.append({
                        'Klassifikator': translate_classifier(classifier),
                        'Strategie': translate_strategy(strategy),
                        'Budget (%)': int(budget_pct * 100),
                        'Verbesserung (%)': improvement
                    })
    
    imp_df = pd.DataFrame(improvements)
    
    # Visualisierung
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Pivot für bessere Darstellung
    pivot_df = imp_df.pivot_table(
        values='Verbesserung (%)',
        index=['Klassifikator', 'Budget (%)'],
        columns='Strategie'
    )
    
    # Heatmap
    sns.heatmap(pivot_df, 
                annot=True, 
                fmt='.2f', 
                cmap='RdYlGn',
                center=0,
                cbar_kws={'label': 'Verbesserung (%)'},
                linewidths=0.5)
    
    plt.title('Prozentuale Verbesserung gegenüber Zufälliger Auswahl', fontsize=14, pad=20)
    plt.xlabel('Strategie', fontsize=12)
    plt.ylabel('Klassifikator / Budget (%)', fontsize=12)
    
    # Speichern
    filename = f'{output_dir}/verbesserung_heatmap_deutsch.png'
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✓ Verbesserungs-Heatmap erstellt: {filename}")
    plt.close()
    
    # Zusätzlich: Balkendiagramm für bessere Sichtbarkeit kleiner Unterschiede
    fig, axes = plt.subplots(1, len(classifiers), figsize=(18, 6))
    
    if len(classifiers) == 1:
        axes = [axes]
    
    for idx, classifier in enumerate([translate_classifier(c) for c in classifiers]):
        ax = axes[idx]
        
        data_for_classifier = imp_df[imp_df['Klassifikator'] == classifier]
        
        # Gruppiert nach Budget
        x = np.arange(len(budget_percentages))
        width = 0.25
        
        for i, strategy in enumerate([translate_strategy(s) for s in strategies]):
            values = []
            for budget in sorted(data_for_classifier['Budget (%)'].unique()):
                val = data_for_classifier[
                    (data_for_classifier['Strategie'] == strategy) & 
                    (data_for_classifier['Budget (%)'] == budget)
                ]['Verbesserung (%)'].values
                values.append(val[0] if len(val) > 0 else 0)
            
            ax.bar(x + i*width - width, values, width, label=strategy, alpha=0.8)
        
        ax.set_xlabel('Budget (%)')
        ax.set_ylabel('Verbesserung (%)')
        ax.set_title(f'{classifier}')
        ax.set_xticks(x)
        ax.set_xticklabels([f'{int(b*100)}%' for b in budget_percentages])
        ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        ax.grid(True, alpha=0.3, axis='y')
        
        if idx == 0:
            ax.legend()
        
        # Angepasste Y-Achse
        y_data = [v for v in values if v != 0]
        if y_data:
            y_min, y_max = calculate_y_limits(y_data, margin=0.2)
            ax.set_ylim(y_min, y_max)
    
    plt.suptitle('Detaillierte Verbesserungsanalyse pro Klassifikator', fontsize=14)
    plt.tight_layout()
    
    filename = f'{output_dir}/verbesserung_balken_deutsch.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✓ Verbesserungs-Balkendiagramm erstellt: {filename}")
    plt.close()

# -------------------------------------------------------------------------------
# 6. Label-Einsparungen visualisieren
# -------------------------------------------------------------------------------

def plot_label_savings_german(savings_df, output_dir='plots'):
    """Visualisiert die Label-Einsparungen auf Deutsch"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('MNIST - Label-Einsparungs-Analyse', fontsize=16)
    
    # 1. Labels benötigt für verschiedene Performance-Level
    ax1 = axes[0, 0]
    
    # Daten für 95% Performance
    data_95 = savings_df[savings_df['target_performance'] == 95]
    
    classifiers = sorted(data_95['classifier'].unique())
    strategies = sorted(data_95['strategy'].unique())
    
    x = np.arange(len(strategies))
    width = 0.25
    
    for i, classifier in enumerate(classifiers):
        values = []
        errors = []
        for strategy in strategies:
            row = data_95[(data_95['classifier'] == classifier) & 
                         (data_95['strategy'] == strategy)]
            if not row.empty:
                values.append(row['avg_labels_needed'].values[0])
                errors.append(row['std_labels_needed'].values[0])
            else:
                values.append(0)
                errors.append(0)
        
        ax1.bar(x + i*width - width, values, width, 
               yerr=errors, capsize=5,
               label=translate_classifier(classifier), alpha=0.8)
    
    ax1.set_xlabel('Strategie')
    ax1.set_ylabel('Benötigte Labels')
    ax1.set_title('Labels für 95% der Baseline-Leistung')
    ax1.set_xticks(x)
    ax1.set_xticklabels([translate_strategy(s) for s in strategies], rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.axhline(y=60000, color='red', linestyle='--', alpha=0.5, label='Vollständiger Datensatz')
    
    # 2. Relative Einsparung gegenüber Random Sampling
    ax2 = axes[0, 1]
    
    # Nur Active Learning Strategien
    al_strategies = [s for s in strategies if s != 'Random Sampling']
    savings_matrix = []
    
    for classifier in classifiers:
        row = []
        for strategy in al_strategies:
            data = savings_df[(savings_df['classifier'] == classifier) & 
                            (savings_df['strategy'] == strategy) & 
                            (savings_df['target_performance'] == 95)]
            if not data.empty:
                row.append(data['relative_savings_pct'].values[0])
            else:
                row.append(0)
        savings_matrix.append(row)
    
    im = ax2.imshow(savings_matrix, cmap='RdYlGn', aspect='auto')
    ax2.set_xticks(range(len(al_strategies)))
    ax2.set_xticklabels([translate_strategy(s) for s in al_strategies], rotation=45, ha='right')
    ax2.set_yticks(range(len(classifiers)))
    ax2.set_yticklabels([translate_classifier(c) for c in classifiers])
    ax2.set_title('Einsparung vs. Zufällige Auswahl (%)')
    
    # Werte in die Zellen schreiben
    for i in range(len(classifiers)):
        for j in range(len(al_strategies)):
            text = ax2.text(j, i, f'{savings_matrix[i][j]:.1f}%',
                           ha="center", va="center", color="black", fontsize=10)
    
    plt.colorbar(im, ax=ax2, label='Einsparung (%)')
    
    # 3. Einsparung nach Performance-Level
    ax3 = axes[1, 0]
    
    performance_levels = sorted(savings_df['target_performance'].unique())
    
    for classifier in classifiers:
        values = []
        for perf_level in performance_levels:
            # Durchschnittliche Einsparung über alle AL-Strategien
            data = savings_df[(savings_df['classifier'] == classifier) & 
                            (savings_df['target_performance'] == perf_level) & 
                            (savings_df['strategy'] != 'Random Sampling')]
            if not data.empty:
                values.append(data['savings_pct'].mean())
            else:
                values.append(0)
        
        ax3.plot(performance_levels, values, marker='o', linewidth=2, 
                label=translate_classifier(classifier))
    
    ax3.set_xlabel('Ziel-Performance (% der Baseline)')
    ax3.set_ylabel('Durchschnittliche Label-Einsparung (%)')
    ax3.set_title('Label-Einsparung nach Ziel-Performance')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Zusammenfassungstabelle
    ax4 = axes[1, 1]
    ax4.axis('tight')
    ax4.axis('off')
    
    # Erstelle Zusammenfassungstabelle für 95% Performance
    summary_data = []
    for classifier in classifiers:
        best_strategy = None
        best_saving = -1
        
        for strategy in al_strategies:
            data = data_95[(data_95['classifier'] == classifier) & 
                          (data_95['strategy'] == strategy)]
            if not data.empty and data['savings_pct'].values[0] > best_saving:
                best_saving = data['savings_pct'].values[0]
                best_strategy = strategy
        
        if best_strategy:
            labels_data = data_95[(data_95['classifier'] == classifier) & 
                                (data_95['strategy'] == best_strategy)]
            labels_needed = int(labels_data['avg_labels_needed'].values[0])
            
            summary_data.append([
                translate_classifier(classifier),
                translate_strategy(best_strategy),
                f'{labels_needed:,}',
                f'{best_saving:.1f}%'
            ])
    
    table = ax4.table(cellText=summary_data,
                     colLabels=['Klassifikator', 'Beste Strategie', 'Labels benötigt', 'Einsparung'],
                     cellLoc='center',
                     loc='center')
    
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 2)
    
    # Header-Zeile formatieren
    for i in range(4):
        table[(0, i)].set_facecolor('#40466e')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    ax4.set_title('Zusammenfassung: Beste Strategien bei 95% Performance', pad=20)
    
    plt.tight_layout()
    
    filename = f'{output_dir}/label_einsparungen_deutsch.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✓ Label-Einsparungs-Analyse erstellt: {filename}")
    plt.close()

# -------------------------------------------------------------------------------
# 7. Statistische Zusammenfassung
# -------------------------------------------------------------------------------

def plot_statistical_summary_german(stat_results, output_dir='plots'):
    """Erstellt eine statistische Zusammenfassung auf Deutsch"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Statistische Analyse - Zusammenfassung', fontsize=16)
    
    # 1. Heatmap der p-Werte
    ax1 = axes[0, 0]
    
    # Pivot-Tabelle erstellen
    pivot_p = pd.pivot_table(
        stat_results,
        values='p_value_corrected',
        index='strategy',
        columns='budget_pct',
        aggfunc='mean'
    )
    
    # Übersetze Strategien
    pivot_p.index = [translate_strategy(s) for s in pivot_p.index]
    
    sns.heatmap(pivot_p, 
                annot=True, 
                fmt='.3f', 
                cmap='RdYlGn_r',
                vmin=0, 
                vmax=0.1,
                cbar_kws={'label': 'Korrigierter p-Wert'},
                ax=ax1)
    
    ax1.set_title('Durchschnittliche p-Werte (Wilcoxon Test)')
    ax1.set_xlabel('Budget (%)')
    ax1.set_ylabel('Strategie')
    
    # 2. Effektstärken
    ax2 = axes[0, 1]
    
    pivot_effect = pd.pivot_table(
        stat_results,
        values='cliffs_delta',
        index='strategy',
        columns='budget_pct',
        aggfunc='mean'
    )
    
    pivot_effect.index = [translate_strategy(s) for s in pivot_effect.index]
    
    sns.heatmap(pivot_effect, 
                annot=True, 
                fmt='.3f', 
                cmap='coolwarm',
                center=0,
                vmin=-0.5, 
                vmax=0.5,
                cbar_kws={'label': "Cliff's Delta"},
                ax=ax2)
    
    ax2.set_title("Durchschnittliche Effektstärke")
    ax2.set_xlabel('Budget (%)')
    ax2.set_ylabel('Strategie')
    
    # 3. Signifikante Verbesserungen
    ax3 = axes[1, 0]
    
    sig_results = stat_results[stat_results['significant']]
    if not sig_results.empty:
        sig_counts = sig_results.groupby('strategy').size()
        sig_counts.index = [translate_strategy(s) for s in sig_counts.index]
        sig_counts.plot(kind='bar', ax=ax3, color='steelblue')
        
        ax3.set_title('Anzahl signifikanter Verbesserungen')
        ax3.set_xlabel('Strategie')
        ax3.set_ylabel('Anzahl')
        ax3.grid(axis='y', alpha=0.3)
    
    # 4. Effektstärke nach Klassifikator
    ax4 = axes[1, 1]
    
    classifier_effects = stat_results.groupby('classifier')['cliffs_delta'].agg(['mean', 'std'])
    classifier_effects.index = [translate_classifier(c) for c in classifier_effects.index]
    
    classifier_effects['mean'].plot(kind='bar', ax=ax4, yerr=classifier_effects['std'], 
                                   capsize=5, color='darkorange')
    ax4.set_title("Durchschnittliche Effektstärke pro Klassifikator")
    ax4.set_xlabel('Klassifikator')
    ax4.set_ylabel("Cliff's Delta (Durchschnitt)")
    ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax4.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    filename = f'{output_dir}/statistische_zusammenfassung_deutsch.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✓ Statistische Zusammenfassung erstellt: {filename}")
    plt.close()

# -------------------------------------------------------------------------------
# 8. Hauptfunktion zum Erstellen aller Plots
# -------------------------------------------------------------------------------

def create_all_plots():
    """Erstellt alle Plots mit deutschen Beschriftungen und optimaler Skalierung"""
    
    print("\nErstelle Visualisierungen...")
    print("-"*60)
    
    # Ausgabeordner
    output_dir = 'plots_deutsch'
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"✓ Ausgabeordner erstellt: {output_dir}")
    
    # 1. Lernkurven
    print("\n1. Lernkurven pro Klassifikator...")
    plot_learning_curves_german(results_df, stat_results, output_dir)
    
    # 2. Verbesserungsanalyse
    print("\n2. Verbesserungsanalyse...")
    plot_improvement_german(results_df, output_dir)
    
    # 3. Label-Einsparungen
    print("\n3. Label-Einsparungs-Analyse...")
    plot_label_savings_german(savings_df, output_dir)
    
    # 4. Statistische Zusammenfassung
    print("\n4. Statistische Zusammenfassung...")
    plot_statistical_summary_german(stat_results, output_dir)
    
    print("\n" + "="*60)
    print("✓ Alle Visualisierungen erfolgreich erstellt!")
    print(f"✓ Ausgabeordner: {output_dir}")
    print("="*60)

# -------------------------------------------------------------------------------
# 9. Zusätzliche Analyse: Zusammenfassende Statistiken
# -------------------------------------------------------------------------------

def print_summary_statistics():
    """Gibt eine Zusammenfassung der wichtigsten Ergebnisse aus"""
    
    print("\n" + "="*60)
    print("ZUSAMMENFASSUNG DER ERGEBNISSE")
    print("="*60)
    
    # Beste Kombinationen bei 100% Budget
    final_results = results_df[results_df['budget_pct'] == 1.0]
    
    print("\nBeste Kombinationen bei 100% Budget:")
    print("-"*60)
    
    summary = final_results.groupby(['classifier', 'strategy'])['accuracy'].agg(['mean', 'std'])
    summary = summary.sort_values('mean', ascending=False).head(10)
    
    for (classifier, strategy), row in summary.iterrows():
        print(f"{translate_classifier(classifier):20} + {translate_strategy(strategy):20} = "
              f"{row['mean']:.4f} (±{row['std']:.4f})")
    
    # Durchschnittliche Verbesserung pro Strategie
    print("\nDurchschnittliche Verbesserung über Random Sampling:")
    print("-"*60)
    
    if not stat_results.empty:
        avg_improvement = stat_results.groupby('strategy')['improvement_pct'].mean()
        for strategy, improvement in avg_improvement.items():
            print(f"{translate_strategy(strategy):20}: {improvement:+.2f}%")
    
    # Label-Einsparungen bei 95% Performance
    print("\nLabel-Einsparungen bei 95% Performance:")
    print("-"*60)
    
    savings_95 = savings_df[savings_df['target_performance'] == 95]
    for classifier in sorted(savings_95['classifier'].unique()):
        classifier_data = savings_95[savings_95['classifier'] == classifier]
        best_saving = classifier_data.loc[classifier_data['savings_pct'].idxmax()]
        print(f"{translate_classifier(classifier):20}: {translate_strategy(best_saving['strategy']):20} "
              f"spart {best_saving['savings_pct']:.1f}% der Labels")

# -------------------------------------------------------------------------------
# Ausführung
# -------------------------------------------------------------------------------

if __name__ == "__main__":
    # Erstelle alle Plots
    create_all_plots()
    
    # Zeige Zusammenfassung
    print_summary_statistics()

Active Learning Visualisierungen - Deutsche Version
Lade Ergebnisse...
✓ Hauptergebnisse geladen: 180 Einträge
✓ Statistische Analyse geladen: 45 Vergleiche
✓ Label-Einsparungen geladen: 36 Einträge

Erstelle Visualisierungen...
------------------------------------------------------------
✓ Ausgabeordner erstellt: plots_deutsch

1. Lernkurven pro Klassifikator...
✓ Lernkurven für CNN erstellt: plots_deutsch/cnn_lernkurven_deutsch.png
✓ Lernkurven für Naive Bayes erstellt: plots_deutsch/naive_bayes_lernkurven_deutsch.png
✓ Lernkurven für Random Forest erstellt: plots_deutsch/random_forest_lernkurven_deutsch.png

2. Verbesserungsanalyse...
✓ Verbesserungs-Heatmap erstellt: plots_deutsch/verbesserung_heatmap_deutsch.png
✓ Verbesserungs-Balkendiagramm erstellt: plots_deutsch/verbesserung_balken_deutsch.png

3. Label-Einsparungs-Analyse...
✓ Label-Einsparungs-Analyse erstellt: plots_deutsch/label_einsparungen_deutsch.png

4. Statistische Zusammenfassung...
✓ Statistische Zusammenfassung ers